In [ ]:
%matplotlib inline

## Aims of this session

- Introduction to registration techniques
- Introduction to optimisers
- Learn how to use gVirtualXRay and CMA-ES to register sample location

## Introduction to simulated real experiments

As you have seen, gVXR provides a strong basis for running x-ray simulations. With the speed provided of GPU acceleration, we are able to quickly produce accurate images.
However, X-ray scanners are never perfect, and a large amount of facts influence the final result. We are able to try and paramaterise these scanning artefacts, and with the use of rapid simulations, obtain images closer to the ground truth.

Some examples of artefacts are polychromatic beam spectra, material density, detector energy response, line spread functions, etc. In this session we will take a look at sample positioning as a use case, but the same approach can be used for all methods.

<!-- 
- gvxr is able to account for most real artefact sources
- therefore we should be able to recreate real experiment scenarios
- however, we can't know a bunch of paramaters just from scans, inc exact sample positioning
- with the strength of fast simulations, we can perform optimisation loops to discovered these paramaters -->

## Quick explanation of registration with CMA-ES stochastic optimizer

Image Registration in xCT is a method to align images on the same set of coordinates.


- What is registration
- Quick mention of 'scoring' images
- Optimiser changes paramaters to get a reduced (error) score
- What is Covariance Matrix Adaption-Evolution Strategy?
    - https://arxiv.org/abs/1604.00772
- What does this mean for us?

## Task 2: Using gvxr, register a plexiglass cube to locate its position
- We know how to change the position of models in gvxr, use cma-es to adjust these paramaters
- As a task, create your own scoring function with help from the previous task
- How fast does yours converge? is it a good result?

- plexiglass data has a lot of post-processing due to using a medical device
    - we have no idea what these methods are
    - includes a sharpening filter, hence the edges

**Main contributors:** F. P. Vidal and Jenna Allsup

**Purpose:** In this notebook, we aim to demonstrate that gVirtualXRay is able to generate analytic simulations on GPU comparable to X-ray radiographs taken with a clinically utilised equipment. Such devices often produce images in negative and image filtering, such as image sharpening, may alter their appearance. Another issue is the lack of flat-field correction to account for variations in the pixel-to-pixel sensitivity of the detector. 

**Material and Methods:** We simulate an image with gVirtualXRay and compare it with a ground truth image. For this purpose, we use a real radiograph of a plexiglass (PMMA) block. Its size is 32x27x4cm. The simulated geometry is registered using numerical optimisation so that its location and orientation match those of the real PMMA block. The source-to-object distance (SOD) and the source-to-detector distance (SDD) are also optimised. The initial estimation of the SDD is 130cm. The beam spectrum is polychromatic and the energy response of the detector is considered. The pixel-to-pixel sensitivity of the detector is corrected in the real radiograph. A plausible image sharpening filter is also integrated and optimised to mimic the appearance of a real image. Photon noise is also added to increase realism. The amount of noise is estimated for the real radiograph and optimised in the simulated image.

**Results:** The [mean absolute percentage error (MAPE)](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error), also known as mean absolute percentage deviation (MAPD), between the real radiograph and the simulated image without noise and with noise. The **[zero-mean normalised cross-correlation](https://en.wikipedia.org/wiki/Cross-correlation#Zero-normalized_cross-correlation_(ZNCC)) and **[Structural Similarity Index (SSIM)](https://en.wikipedia.org/wiki/Structural_similarity)** are also used.

The **ZNCC** between the experimental digital radiograph and the simulated image is **98.56%**. The value is extremely close to 100%. The **SSIM** is **0.93**. The value is extremely close to 1.0. These results show that the simulated image is similar to the real X-ray radiograph acquired with a medical device. MAPE is, however, **10.84%**. The difference between the real X-ray radiograph and the simulated image is due to the sharpening filter (we do not actually know how the manufacturer implemented it) and the pixel-to-pixel sensitivity of the detector (even after correction the horizontal profile in the middle of the real X-ray radiograph is not flat).

# Import packages

- Pretty common stuff: re, sys, os, math, copy, numpy
- cma for optimisation (minimisation of an objective function)
- gvxrPython3 to simulate X-ray images
- imageio to generate a GIF file
- matplotlib to plot graphs and display images
- scikit-image for comparing images and for implementing the image sharpening filter
- scikit-learn for computing the image comparison
- scipy for resampling the beam spectrum
- SimpleITK to load the medical DICOM file
- spekpy to generate a beam spectrum
- tifffile to load and save TIFF images in Float32

In [ ]:
import os, math

import numpy as np # Who does not use Numpy?

import matplotlib # Plot graphs and display images

font = {'family' : 'serif',
        #'weight' : 'bold',
         'size'   : 12.5
       }
matplotlib.rc('font', **font)

import matplotlib.pyplot as plt # Plotting

from tifffile import imwrite, imread # Load and save TIFF images in Float32
import imageio # Generate a GIF file

import SimpleITK as sitk # Load the medical DICOM file

import cma # Optimisation (minimisation of an objective function)

from gvxrPython3.utils import compareWithGroundTruth # Plot the ground truth, the test image, and the relative error map in %

from gvxrPython3 import gvxr # Simulate X-ray images
from gvxrPython3.utils import loadSpectrumXpecgen # Generate and load an X-ray spectrum using xpecgen
from IPython.display import display, Image
from skimage.util import compare_images

# import re, sys, os, math, copy


# # old_backend =  matplotlib.get_backend() 
# # matplotlib.use("Agg")  # Prevent showing stuff

# from matplotlib.cm import get_cmap
# from matplotlib.colors import LogNorm # Look up table
# from matplotlib.colors import PowerNorm # Look up table
# import matplotlib.colors as mcolors

# matplotlib.rc('text', usetex=True)

from skimage.filters import gaussian # Implementing the image sharpening filter
from skimage.metrics import structural_similarity as ssim # Quantifying the similarities between the simulated image and the ground truth
# # from sklearn.metrics import mean_squared_error # For the objective function
# # from sklearn.metrics import mean_absolute_percentage_error as mape # Quantifying the difference between the simulated image and the ground truth
# from scipy import signal # Resampling the beam spectrum
# # import spekpy as sp # Generate a beam spectrum

import datetime # For the runtime

# from utils import *

Make sure the output directory exists

In [ ]:
# The directory does not exist
if not os.path.exists("output_data"):
    
    # Create the directory
    os.mkdir("output_data")

# Preparation of the ground truth image

## Read the real X-ray radiograph from a DICOM file

In [ ]:
reader = sitk.ImageFileReader()
reader.SetImageIO("GDCMImageIO")
reader.SetFileName("input_data/PMMA_block/DX000000")
reader.LoadPrivateTagsOn()
reader.ReadImageInformation()    
volume = reader.Execute()
raw_real_image = sitk.GetArrayFromImage(volume)[0]

## Extract the image size and pixel spacing from the DICOM file

It will be useful to set the X-ray detector parameters for the simulation, and to display the images in millimetres.

In [ ]:
spacing = volume.GetSpacing()[0:2]
size = volume.GetSize()[0:2]

print("Image size:", size, "[px]")
print("Pixel spacing:", spacing, "[mm]")

## Extract the kVp from the DICOM file

It will be useful to generate a realistic beam spectrum.

In [ ]:
kVp = float(volume.GetMetaData("0018|0060"))
print("Peak kilo voltage output of the x-ray generator used: ", kVp)

## Display the experimental radiograph from the DICOM file

In [ ]:
plt.figure(figsize= (20,10))
xrange=range(raw_real_image.shape[1])
yrange=range(raw_real_image.shape[0])

plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.title("X-ray image\nkVp:" + str(kVp))
plt.imshow(raw_real_image, cmap="gray", 
           vmin=3934, vmax = 11045,
           extent=[0,(raw_real_image.shape[1]-1)*spacing[0],0,(raw_real_image.shape[0]-1)*spacing[1]])
plt.colorbar(orientation='vertical')

plt.savefig('output_data/07-PMMA_experimental_image.pdf')
plt.savefig('output_data/07-PMMA_experimental_image.png')

In [ ]:
plt.figure(figsize= (20,10))

plt.xlabel("Pixel position\n(in px)")
plt.ylabel("Pixel intensity")
plt.title("Horizontal profile\n(middle row of the image)")
plt.plot(raw_real_image[raw_real_image.shape[0] // 2])
# plt.colorbar(orientation='vertical')

plt.savefig('output_data/07-PMMA_experimental_image-horizontal_profile.pdf')
plt.savefig('output_data/07-PMMA_experimental_image-horizontal_profile.png')

The image is displayed in negative (low attenuation in black, high attenuation in white). We can see on the pictures above that the pixel-to-pixel sensitivity of the detector varries. Pixels on the left-hand side of the image are brighter than on the right-hand side. On the top of that, a sharpening filter is applied to enhance edges. This is common on medical devices to automatially apply image post-processing filters. A collimator (lead plates) is also used to limit the X-ray beam on the PMMA block only.

## Flat-field correction

We are going to correct the pixel-to-pixel sensitivity of the detector and implement a mock flat-field correction. We extract two lines from the image where there is no object. The two lines are stored in two 1D arrays. We create a new array as the element-wise average of the two arrays. Every line of the image is then corrected with an element-wise division.

In [ ]:
line1 = raw_real_image[222,:] # A line of the image with no object
line2 = raw_real_image[2283,:] # Another line of the image with no object
line = 0.5 * line1 + 0.5 * line2 # Element-wise average of the two arrays
line[line.shape[0]-1] = 1 # Make sure the last element is not zero (to avoid a division by zero)

# Create the full-field image
full_field = np.zeros(raw_real_image.shape)

for i in range(raw_real_image.shape[1]):
    full_field[:,i] = line[i]

# Apply the correction
real_image = raw_real_image / full_field

# Save the corrected image
imwrite("output_data/07-PMMA_block_real_image_flat.tif", real_image.astype(np.single))

Display the image after correction

In [ ]:
plt.figure(figsize= (20,10))
xrange=range(real_image.shape[1])
yrange=range(real_image.shape[0])

plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.title("X-ray image\nkVp:" + str(kVp))
plt.imshow(real_image, cmap="gray", 
           # vmin=-1, vmax = 1,
           extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
plt.colorbar(orientation='vertical')

plt.savefig('output_data/07-PMMA_experimental_image_flat.pdf')
plt.savefig('output_data/07-PMMA_experimental_image_flat.png')

# Define a function for the zero-mean, unit-variance normalisation

In [ ]:
def standardisation(img:np.array) -> np.array:
    return (img - img.mean()) / img.std()

## Crop it to remove the text

There are parts of the image that need to be removed, e.g. the text.

In [ ]:
plt.figure(figsize= (20,10))

roi = [179, 2300, 0, 2848]
roi_real_image = standardisation(real_image[179:2300,0:2848])
roi_raw_real_image = standardisation(real_image[179:2300,0:2848])

xrange=range(roi_real_image.shape[1])
yrange=range(roi_real_image.shape[0])

plt.xlabel("Pixel position\n(in mm)")
plt.ylabel("Pixel position\n(in mm)")
plt.title("X-ray image\nkVp:" + str(kVp))
plt.imshow(roi_real_image, cmap="gray", 
           extent=[roi[2] * spacing[0], (roi[3] -1) *spacing[0], roi[0]*spacing[0],(roi[1] - 1)*spacing[0]])
plt.colorbar(orientation='vertical')

plt.savefig('output_data/07-PMMA_experimental_image-cropped.pdf')
plt.savefig('output_data/07-PMMA_experimental_image-cropped.png')

# Initialise gVirtualXRay

## Set the experimental parameters (e.g. source and detector positions, etc.)

We use known parameters as much as possible, for example we know the size and composition of the sample. Some parameters are extracted from the DICOM file, such as detector size, pixel resolution, and voltage of the X-ray tube.

In [ ]:
source_detector_distance_in_cm = 130  # See email Mon 05/07/2021 15:29
block_width_in_cm = 32
block_height_in_cm = 27
block_thickness_in_cm = 4

window_size =  [800, 450]
source_position = [0.0, 0.0, source_detector_distance_in_cm - block_thickness_in_cm / 2, "cm"]
detector_position = [0.0, 0.0, - block_thickness_in_cm / 2, "cm"]
detector_up = [0, 1, 0]

## Initialise the simulation engine

In [ ]:
# Create an OpenGL context
print("Create an OpenGL context:",
    str(window_size[0]) + "x" + str(window_size[1])
)

gvxr.createWindow(-1, True, "EGL")

gvxr.setWindowSize(
    window_size[0],
    window_size[1]
)

## Function to create a PMMA block

In [ ]:
def createBlock(x, y, z, r, h):
    # Remove all the geometries from the whole scenegraph
    gvxr.removePolygonMeshesFromSceneGraph()

    # Make a cube
    gvxr.makeCube("PMMA block", 1.0, "mm")

    # Translation vector
    gvxr.translateNode("PMMA block", x, y, z, "cm")

    # Rotation angle
    gvxr.rotateNode("PMMA block", r, 0, 0, 1)

    # Scaling factors
    gvxr.scaleNode("PMMA block", block_width_in_cm * 10, h * 10, block_thickness_in_cm * 10);

    # Apply the transformation matrix so that we can save the corresponding STL file
    gvxr.applyCurrentLocalTransformation("PMMA block");

    # Set the matrix's material properties
    gvxr.setCompound("PMMA block", "C5O2H8")
    gvxr.setDensity("PMMA block", 1.18, "g/cm3")

    # Add the matrix to the X-ray renderer
    gvxr.addPolygonMeshAsInnerSurface("PMMA block")
    
    # Change the colour
    gvxr.setColour("PMMA block", 0.5, 0.5, 0, 1)

In [ ]:
# Initial guess
x = y = 0
z = 0
r = 0
createBlock(x, y, z, r, block_height_in_cm)

## Set the source position

In [ ]:
# Set up the beam
print("Set up the beam")
print("\tSource position:", source_position)
gvxr.setSourcePosition(
    source_position[0],
    source_position[1],
    source_position[2],
    source_position[3]
);

gvxr.usePointSource();

focal_spot_size = volume.GetMetaData("0018|1413")
# gvxr.setFocalSpot(focal_spot_size, focal_spot_size, focal_spot_size, "mm");
print("size of focal spot (in mm):", focal_spot_size)

## Get the spectrum from the DICOM file

In [ ]:
spectrum, k, f, unit = loadSpectrumXpecgen(kVp, [["Al", 3]])

In [ ]:
plt.figure()
plt.plot(k,f)
plt.title("Beam spectrum\n(kVp: " + str(kVp) + ", filtration: 3mm of Al)")
plt.xlabel("Energy [keV]")
plt.ylabel("Normalised number of photons")
plt.grid(True)

## Set the X-ray detector

In [ ]:
# Set up the detector
print("Set up the detector");
print("\tDetector position:", detector_position)
gvxr.setDetectorPosition(
    detector_position[0],
    detector_position[1],
    detector_position[2],
    detector_position[3]
);

print("\tDetector up vector:", detector_up)
gvxr.setDetectorUpVector(
    detector_up[0],
    detector_up[1],
    detector_up[2]
);

In [ ]:
print("\tDetector number of pixels:", size)
gvxr.setDetectorNumberOfPixels(
    size[0],
    size[1]
);

print("\tPixel spacing:", spacing)
gvxr.setDetectorPixelSize(
    spacing[0],
    spacing[1],
    "mm"
);

Load the detector response in energy

In [ ]:
gvxr.clearDetectorEnergyResponse() # To be on the safe side
gvxr.loadDetectorEnergyResponse("input_data/responseDetector.txt",
                                "MeV")


## Take a screenshot of the 3D environment

In [ ]:
gvxr.displayScene()

gvxr.useNegative()
gvxr.useLighing()
gvxr.useWireframe(False)
gvxr.setSceneRotationMatrix([0.43813619017601013, 0.09238918125629425, -0.8941444158554077, 0.0,
                             0.06627026945352554, 0.9886708855628967, 0.13463231921195984, 0.0,
                             0.8964602947235107, -0.11824299395084381, 0.4270564019680023, 0.0,
                             0.0, 0.0, 0.0, 1.0])
gvxr.setZoom(1639.6787109375)

gvxr.displayScene()

In [ ]:
screenshot = gvxr.takeScreenshot()

In [ ]:
plt.figure(figsize= (10,10))
plt.title("Screenshot")
plt.imshow(screenshot)
plt.axis('off')

plt.tight_layout()

plt.savefig('output_data/07-PMMA_screenshot-beam-off.pdf')
plt.savefig('output_data/07-PMMA_screenshot-beam-off.png')

In [ ]:
gvxr.computeXRayImage()
gvxr.displayScene()

In [ ]:
screenshot = gvxr.takeScreenshot()

plt.figure(figsize= (10,10))
plt.title("Screenshot")
plt.imshow(screenshot)
plt.axis('off')

plt.tight_layout()

plt.savefig('output_data/07-PMMA_screenshot-beam-on.pdf')
plt.savefig('output_data/07-PMMA_screenshot-beam-on.png')

# Optimise the position and orientation of the PMMA block, and refine the SDD

## Create an objective function

In [ ]:
def zncc(i1, i2):
    return (np.mean(np.multiply(i1, i2))) / 2.0;

In [ ]:
def mean_squared_error(img1: np.array, img2: np.array) -> float:
    return np.mean(np.square(img1 - img2))

In [ ]:
def objectiveFunction(parameters):
    
    global best_fitness
    global best_fitness_id
    global fitness_function_call_id
    global evolution_zncc
    global evolution_parameters
    
    # Retrieve the parameters
    x, y, z, r, h, SDD = parameters
    
    # Update the source position
    source_position = [0.0, 0.0, SDD - block_thickness_in_cm / 2, "cm"]
    gvxr.setSourcePosition(
        source_position[0],
        source_position[1],
        source_position[2],
        source_position[3]
    );

    # Update the block geometry
    createBlock(x, y, z, r, h)
    
    # Compute an X-ray image
    x_ray_image = np.array(gvxr.computeXRayImage())
    
    # Compute the negative image as it is the case for the real image
    x_ray_image *= -1.
    
    # Crop the image
    x_ray_image = x_ray_image[179:2300,0:2848]

    # Zero-mean, unit-variance normalistion
    x_ray_image = standardisation(x_ray_image)
    
    # Return the objective
    objective = math.sqrt(mean_squared_error(roi_real_image, x_ray_image))

    # The block below is not necessary for the registration.
    # It is used to save the data to create animations.
    if best_fitness > objective:
        
        gvxr.displayScene()
        screenshot = (255 * np.array(gvxr.takeScreenshot())).astype(np.uint8)

        # gvxr.saveSTLfile("PMMA block", "gVirtualXRay_output_data/PMMA_block_" + str(best_fitness_id) + ".stl")
        # np.savetxt("gVirtualXRay_output_data/PMMA_block_" + str(best_fitness_id) + ".dat", [x, y, z, r, h, SDD], header='x,y,z,r,h,SDD')
        imwrite("output_data/07-PMMA_block_xray_" + str(best_fitness_id) + ".tif", x_ray_image.astype(np.single))
        imwrite("output_data/07-PMMA_block_screenshot_" + str(best_fitness_id) + ".tif", screenshot)
    
        zncc_value = zncc(roi_real_image, x_ray_image)
        evolution_zncc.append([fitness_function_call_id, zncc_value])
        evolution_parameters.append([fitness_function_call_id, x, y, z, r, h, SDD])
        
        best_fitness = objective
        best_fitness_id += 1
    
    fitness_function_call_id += 1
    
    return objective

In [ ]:
# Zero-mean, unit-variance normalistion
roi_real_image = standardisation(roi_real_image)
imwrite("output_data/07-PMMA_block_roi_real_image.tif", roi_real_image)

In [ ]:
# The registration has already been performed. Load the results.
if os.path.isfile("output_data/07-PMMA_block.dat") and \
        os.path.isfile("output_data/07-PMMA_block_evolution_zncc.dat") and \
        os.path.isfile("output_data/07-PMMA_block_evolution_parameters.dat"):
    
    temp = np.loadtxt("output_data/07-PMMA_block.dat")
    x = temp[0]
    y = temp[1]
    z = temp[2]
    r = temp[3]
    block_height_in_cm = temp[4]
    source_detector_distance_in_cm = temp[5]
    
    # Update the source position
    source_position = [0.0, 0.0, source_detector_distance_in_cm - block_thickness_in_cm / 2, "cm"]
    gvxr.setSourcePosition(
        source_position[0],
        source_position[1],
        source_position[2],
        source_position[3]
    );
    
    # Update the block geometry
    createBlock(x, y, z, r, block_height_in_cm)

    evolution_zncc = np.loadtxt("output_data/07-PMMA_block_evolution_zncc.dat")
    evolution_parameters = np.loadtxt("output_data/07-PMMA_block_evolution_parameters.dat")

else:
    # Optimise
    opts = cma.CMAOptions()
    opts.set('tolfun', 1e-2)
    opts['tolx'] = 1e-2
    opts['bounds'] = [[-20, -20, detector_position[2] + block_thickness_in_cm / 2, -90, block_height_in_cm * 0.80, source_detector_distance_in_cm * 0.80],
                      [20, 20, source_position[2] - block_thickness_in_cm / 2, 90, block_height_in_cm * 1.20, source_detector_distance_in_cm * 1.20]]
    opts['CMA_stds'] = []

    for min_val, max_val in zip(opts['bounds'][0], opts['bounds'][1]):
        opts['CMA_stds'].append(abs(max_val - min_val) * 0.05)

    best_fitness = sys.float_info.max;
    best_fitness_id = 0;
    fitness_function_call_id = 0
    evolution_zncc = []
    evolution_parameters = []
        
    # Optimise
    res = cma.fmin(objectiveFunction,
              [x, y, z, r, block_height_in_cm, source_detector_distance_in_cm],
              0.5,
              opts) 
    x, y, z, r, block_height_in_cm, source_detector_distance_in_cm = res[0]
    
    # Save the parameters
    np.savetxt("output_data/07-PMMA_block.dat", [x, y, z, r, block_height_in_cm, source_detector_distance_in_cm], header='x,y,z,r,h,SDD')

    # Update the source position
    source_position = [0.0, 0.0, source_detector_distance_in_cm - block_thickness_in_cm / 2, "cm"]
    gvxr.setSourcePosition(
        source_position[0],
        source_position[1],
        source_position[2],
        source_position[3]
    );
    
    # Update the block geometry
    createBlock(x, y, z, r, block_height_in_cm)
    gvxr.saveSTLfile("PMMA block", "output_data/07-PMMA_block.stl")

    evolution_zncc = np.array(evolution_zncc)
    np.savetxt("output_data/07-PMMA_block_evolution_zncc.dat", evolution_zncc, header='t,ZNCC')

    evolution_parameters = np.array(evolution_parameters)
    np.savetxt("output_data/07-PMMA_block_evolution_parameters.dat", evolution_parameters, header='t,x,y,z,r,h,SDD')

In [ ]:
print("SDD (in cm):", source_detector_distance_in_cm)
print("Block centre (in cm):", x, y, z)
print("Block size (in cm):", block_width_in_cm, block_height_in_cm, block_thickness_in_cm)
print("Rotation angle (in degrees):", r)

In [ ]:
evolution_parameters = np.array(evolution_parameters)

plt.figure(figsize= (20,10))
plt.title("Evolution of the PMMA block centre of mass along the three carthesian coordinates")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,1], label="x")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,2], label="y")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,3], label="z")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Position (in cm)")
plt.legend()
plt.savefig('output_data/07-PMMA_block_evolution_centre_of_mass.pdf')
plt.savefig('output_data/07-PMMA_block_volution_centre_of_mass.png')

plt.figure(figsize= (20,10))
plt.title("Evolution of the rotation angle of the PMMA block")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,4], label="angle")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Rotation angle (in degrees)")
plt.savefig('output_data/07-PMMA_block_evolution_centre_of_mass.pdf')
plt.savefig('output_data/07-PMMA_block_evolution_centre_of_mass.png')

plt.figure(figsize= (20,10))
plt.title("Evolution of the height of the PMMA block")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,5], label="angle")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Height (in cm)")
plt.savefig('output_data/07-PMMA_block_evolution_height.pdf')
plt.savefig('output_data/07-PMMA_block_evolution_height.png')

plt.figure(figsize= (20,10))
plt.title("Evolution of the source-detector distance")
plt.plot(evolution_parameters[:,0], evolution_parameters[:,6])
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("Distance (in mm)")
plt.savefig('output_data/07-PMMA_block_evolution_SDD.pdf')
plt.savefig('output_data/07-PMMA_block_evolution_SDD.png')

## Apply the result of the optimisation

In [ ]:
# Compute an X-ray image
raw_x_ray_image = np.array(gvxr.computeXRayImage())

gvxr.displayScene()

# Compute the negative image as it is the case for the real image
x_ray_image = raw_x_ray_image * -1.

# Crop the image
x_ray_image = x_ray_image[179:2300,0:2848]

# Zero-mean, unit-variance normalistion
x_ray_image = standardisation(x_ray_image)

## Plot the evolution of the quality of the simulated image

In [ ]:
plt.figure(figsize= (20,10))
plt.title("Evolution of the ZNCC between the experimental and simulated images")
plt.plot(evolution_zncc[:,0], 100.0 * evolution_zncc[:,1], "go-")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("ZNCC\n(in \%)")
plt.savefig('output_data/07-PMMA_block_evolution_ZNCC1.pdf')
plt.savefig('output_data/07-PMMA_block_evolution_ZNCC1.png')

In [ ]:
def compareImages(real_image, x_ray_image, colorbar=True, rows = 1, cols = 3):
    comp_equalized = compare_images(real_image, x_ray_image, method='checkerboard');
    
#     plt.figure(figsize= (20,10))

    # plt.suptitle("Image simulated using gVirtualXRay without the energy response of the detector", y=1.02)

    plt.subplot(rows, cols, 1)
    plt.imshow(real_image, cmap="gray", vmin=-1, vmax=1,
               extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
    if colorbar:
        plt.colorbar(orientation='horizontal')
    plt.title("Experimental image")
    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Pixel position\n(in mm)")

    plt.subplot(rows, cols, 2)
    plt.imshow(x_ray_image, cmap="gray", vmin=-1, vmax=1,
               extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
    if colorbar:
        plt.colorbar(orientation='horizontal')
    plt.title("Simulated image")
    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Pixel position\n(in mm)")

    plt.subplot(rows, cols, 3)
    plt.imshow(comp_equalized, cmap="gray", vmin=-1, vmax=1,
               extent=[0,(real_image.shape[1]-1)*spacing[0],0,(real_image.shape[0]-1)*spacing[1]])
    if colorbar:
        plt.colorbar(orientation='horizontal')
    plt.title("Checkerboard comparison")
    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Pixel position\n(in mm)")

    plt.tight_layout()

In [ ]:
compareWithGroundTruth(roi_raw_real_image, x_ray_image, figsize=(12.5, 5))

In [ ]:
def drawHorizontalProfiles(real_image, x_ray_image, x_ray_image_noise=None):

    horizontal_profile_real_image = real_image[real_image.shape[1] // 2]
    horizontal_profile_simulated_image = x_ray_image[x_ray_image.shape[1] // 2]

    x_val = np.linspace(0.0,
                        spacing[0] * real_image.shape[1],
                        real_image.shape[1], endpoint=True)

    plt.plot(x_val, horizontal_profile_real_image, label="Experimental image")
    
    if x_ray_image_noise is not None:
        horizontal_profile_simulated_image_noise = x_ray_image_noise[x_ray_image_noise.shape[1] // 2]
        plt.plot(x_val, horizontal_profile_simulated_image_noise, label="Simulated image with noise")

    plt.plot(x_val, horizontal_profile_simulated_image, label="Simulated image without noise")

    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Relative intensity")

In [ ]:
def drawVerticalProfiles(real_image, x_ray_image, x_ray_image_noise=None):
    vertical_profile_real_image = real_image[:,real_image.shape[0] // 2]
    vertical_profile_simulated_image = x_ray_image[:,x_ray_image.shape[0] // 2]

    x_val = np.linspace(0.0,
                        spacing[1] * real_image.shape[0],
                        real_image.shape[0], endpoint=True)

    plt.plot(x_val, vertical_profile_real_image, label="Experimental image")
    
    if x_ray_image_noise is not None:
        vertical_profile_simulated_image_noise = x_ray_image_noise[:,x_ray_image_noise.shape[0] // 2]
        plt.plot(x_val, vertical_profile_simulated_image_noise, label="Simulated image with noise")

    plt.plot(x_val, vertical_profile_simulated_image, label="Simulated image without noise")

    plt.xlabel("Pixel position\n(in mm)")
    plt.ylabel("Relative intensity")

In [ ]:
plt.figure(figsize= (20,10))
drawHorizontalProfiles(roi_real_image, x_ray_image)
plt.legend()
plt.savefig('output_data/07-PMMA_block_compare_horizontal_profile1.pdf')
plt.savefig('output_data/07-PMMA_block_compare_horizontal_profile1.png')

In [ ]:
plt.figure(figsize= (20,10))
drawVerticalProfiles(roi_real_image, x_ray_image)
plt.legend()
plt.savefig('output_data/07-PMMA_block_compare_vertical_profile1.pdf')
plt.savefig('output_data/07-PMMA_block_compare_vertical_profile1.png')

In [ ]:
def createAnimation(evolution_parameters, real_image):
    # Create the GIF file
    with imageio.get_writer("output_data/07-PMMA_block_evolution.gif", mode='I') as writer:

        # Store the PNG filenames
        png_filename_set = [];

        # Process all the images
        for i, [t, x, y, z, r, w, SDD] in enumerate(evolution_parameters):
            t = int(t)
            
            x_ray_image = imread("output_data/07-PMMA_block_xray_" + str(i) + ".tif")
            screenshot = imread("output_data/07-PMMA_block_screenshot_" + str(i) + ".tif")
            
            # Create the figure
            fig, axs = plt.subplots(nrows=2, ncols=3, figsize= (20,10))
            plt.suptitle("Iteration " + str(t+1) + "/" + str(int(evolution_parameters[-1][0]+1)))
            compareImages(real_image, x_ray_image, False, 2, 3)
      
            plt.subplot(234)
            drawHorizontalProfiles(real_image, x_ray_image)

            plt.subplot(235)
            drawVerticalProfiles(real_image, x_ray_image)
  

            plt.subplot(236)
            plt.imshow(screenshot)

            # Save the figure as a PNG file
            plt.savefig("temp.png")

            # Close the figure
            plt.close()

            # Open the PNG file with imageio and add it to the GIF file
            image = imageio.imread("temp.png")
            writer.append_data(image)

            # Delete the PNG file
            os.remove("temp.png");

        for i in range(15):
            writer.append_data(image)

In [ ]:
if not os.path.exists("output_data/07-PMMA_block_evolution.gif"):
    createAnimation(evolution_parameters, roi_real_image)

In [ ]:
with open('output_data/07-PMMA_block_evolution.gif','rb') as f:
    display(Image(data=f.read(), format='png', width=1000))

# Post-processing using image sharpening

We can see from the real image that an image sharpening filter was applied. We will implement one and optimise its parameters.

In [ ]:
def sharpen(image, ksize, alpha, shift, scale):
    details = image - gaussian(image, ksize)    

    return scale * (shift + image) + alpha * details

In [ ]:
def postProcessing(raw_x_ray_image, sigma1, sigma2, alpha, shift, scale):
    # Compute an X-ray image
    x_ray_image = sharpen(raw_x_ray_image, [sigma1, sigma2], alpha, shift, scale)
    
    # Compute the negative image as it is the case for the real image
    x_ray_image *= -1.
    
    # Crop the image
    x_ray_image = x_ray_image[179:2300,0:2848]

    # Zero-mean, unit-variance normalistion
    x_ray_image = standardisation(x_ray_image)
    
    return x_ray_image

## Define an objective function

In [ ]:
def objectiveFunctionSharpen(parameters):
    
    global best_fitness
    global best_fitness_id
    global fitness_function_call_id
    global sharpened_evolution_zncc
    global sharpened_evolution_parameters

    global raw_x_ray_image
    
    # Retrieve the parameters
    sigma1, sigma2, alpha, shift, scale = parameters
    
    # Compute an X-ray image
    x_ray_image = postProcessing(raw_x_ray_image, sigma1, sigma2, alpha, shift, scale)
        
    # Return the objective
    objective = math.sqrt(mean_squared_error(roi_real_image, x_ray_image))

    # The block below is not necessary for the registration.
    # It is used to save the data to create animations.
    if best_fitness > objective:
        
        imwrite("output_data/07-PMMA_block_sharpened_xray_" + str(best_fitness_id) + ".tif", x_ray_image.astype(np.single))
    
        zncc_value = zncc(roi_real_image, x_ray_image)
        sharpened_evolution_zncc.append([fitness_function_call_id, zncc_value])
        sharpened_evolution_parameters.append([fitness_function_call_id, sigma1, sigma2, alpha, shift, scale])
        
        best_fitness = objective
        best_fitness_id += 1
    
    fitness_function_call_id += 1
    
    return objective

## Minimise the objective function

In [ ]:
old_zncc = evolution_zncc[-1][1]

In [ ]:
best_fitness = sys.float_info.max;
best_fitness_id = 0;
fitness_function_call_id = 0
sharpened_evolution_zncc = []
sharpened_evolution_parameters = []

sigma1 = 100
sigma2 = 100
alpha = 2.5
shift = 0
scale = 1

xl = [3, 3, 0, -5, 0]
xu = [200, 200, 15, 5, 2]
x_init = [sigma1, sigma2, alpha, shift, scale]

In [ ]:
# The registration has already been performed. Load the results.
if os.path.isfile("output_data/07-PMMA_block_sharpen.dat") and \
        os.path.isfile("output_data/07-PMMA_block_sharpen_evolution_zncc.dat") and \
        os.path.isfile("output_data/07-PMMA_block_sharpen_evolution_parameters.dat"):

    temp = np.loadtxt("output_data/07-PMMA_block_sharpen.dat")
    sigma1 = temp[0]
    sigma2 = temp[1]
    alpha = temp[2]
    shift = temp[3]
    scale = temp[4]
    
    sharpened_evolution_zncc = np.loadtxt("output_data/07-PMMA_block_sharpen_evolution_zncc.dat")
    sharpened_evolution_parameters = np.loadtxt("output_data/07-PMMA_block_sharpen_evolution_parameters.dat")

else:
    # Optimise
    timeout_in_sec = 20 * 60 # 20 minutes
    opts = cma.CMAOptions()
    opts.set('tolfun', 1e-5)
    opts['tolx'] = 1e-5
    opts['timeout'] = timeout_in_sec
    opts['bounds'] = [xl, xu]
    opts['CMA_stds'] = []

    for min_val, max_val in zip(opts['bounds'][0], opts['bounds'][1]):
        opts['CMA_stds'].append(abs(max_val - min_val) * 0.05)

    # Optimise
    es = cma.CMAEvolutionStrategy(x_init, 0.5, opts)
    es.optimize(objectiveFunctionSharpen)

    # Save the parameters
    sigma1, sigma2, alpha, shift, scale = es.result.xbest
    np.savetxt("output_data/07-PMMA_block_sharpen.dat", [sigma1, sigma2, alpha, shift, scale], header='sigma,alpha,shift,scale')

    sharpened_evolution_zncc = np.array(sharpened_evolution_zncc)
    np.savetxt("output_data/07-PMMA_block_sharpen_evolution_zncc.dat", sharpened_evolution_zncc, header='t,ZNCC')

    sharpened_evolution_parameters = np.array(sharpened_evolution_parameters)
    np.savetxt("output_data/07-PMMA_block_sharpen_evolution_parameters.dat", sharpened_evolution_parameters, header='t,sigma,alpha')

    # Release memory
    del es;

In [ ]:
new_zncc = sharpened_evolution_zncc[-1][1]

In [ ]:
print("ZNCC before sharpening:", str(100 * old_zncc) + "%")
print("ZNCC after sharpening:", str(100 * new_zncc) + "%")

In [ ]:
print(evolution_zncc[-1,0])

In [ ]:
plt.figure(figsize= (20,10))
plt.title("Evolution of the ZNCC between the experimental and simulated images")
plt.plot(evolution_zncc[:,0], 100 * evolution_zncc[:,1], "go-", label="Optimisation of the geometry")
plt.plot(sharpened_evolution_zncc[:,0] + evolution_zncc[-1,0], 100 * sharpened_evolution_zncc[:,1], "ro-", label="Optimisation of the digital imaging filters")
plt.xlabel("Timeline\n(in number of fitness function calls)")
plt.ylabel("ZNCC\n(in \%)")
plt.legend()
plt.savefig('output_data/07-PMMA_block_evolution_ZNCC2.pdf')
plt.savefig('output_data/07-PMMA_block_evolution_ZNCC2.png')

## Apply the result of the optimisation

In [ ]:
x_ray_image = postProcessing(raw_x_ray_image, sigma1, sigma2, alpha, shift, scale)

In [ ]:
compareWithGroundTruth(roi_real_image, x_ray_image, figsize=(12.5, 5))

In [ ]:
plt.figure(figsize= (20,10))
compareImages(roi_real_image, x_ray_image)
plt.savefig('output_data/07-PMMA_block_compare_images2.pdf')
plt.savefig('output_data/07-PMMA_block_compare_images2.png')

# Plot the profiles

In [ ]:
plt.figure(figsize= (20,10))
drawHorizontalProfiles(roi_real_image, x_ray_image)
plt.legend()
plt.savefig('output_data/07-PMMA_block_compare_horizontal_profile2.pdf')
plt.savefig('output_data/07-PMMA_block_compare_horizontal_profile2.png')

In [ ]:
plt.figure(figsize= (20,10))
drawVerticalProfiles(roi_real_image, x_ray_image)
plt.legend()
plt.savefig('output_data/07-PMMA_block_compare_vertical_profile2.pdf')
plt.savefig('output_data/07-PMMA_block_compare_vertical_profile2.png')

# Quantify similarities/dissimilarities

In [ ]:
def mape(ref: np.array, test: np.array) -> float:
    return np.mean(np.abs((ref - test) / ref))

In [ ]:
# Avoid div by 0
offset1 = min(roi_real_image.min(), x_ray_image.min())
offset2 = 0.01 * (roi_real_image.max() - roi_real_image.min())
offset = offset2 - offset1

MAPE = mape(roi_real_image + offset, x_ray_image + offset)
ZNCC = np.mean((roi_real_image - roi_real_image.mean()) / roi_real_image.std() * (x_ray_image - x_ray_image.mean()) / x_ray_image.std())
SSIM = ssim(roi_real_image, x_ray_image, data_range=roi_real_image.max() - roi_real_image.min())

print("MAPE without noise:", "{0:0.2f}".format(100 * MAPE) + "%")
print("ZNCC without noise", "{0:0.2f}".format(100 * ZNCC) + "%")
print("SSIM without noise:", "{0:0.2f}".format(SSIM))

# Estimate the simulation time

Get the total number of triangles

In [ ]:
number_of_triangles = gvxr.getNumberOfPrimitives("PMMA block")

Compute an X-ray image 25 times (to gather performance statistics)

In [ ]:
# gvxr.enableArtefactFilteringOnCPU()
gvxr.enableArtefactFilteringOnGPU()
# gvxr.disableArtefactFiltering() # Spere inserts are missing with GPU integration when a outer surface is used for the matrix

runtimes = []

for i in range(25):
    start_time = datetime.datetime.now()

    raw_x_ray_image = np.array(gvxr.computeXRayImage())

    # Compute and post-process an X-ray image
    x_ray_image = sharpen(raw_x_ray_image, [sigma1, sigma2], alpha, shift, scale)

    end_time = datetime.datetime.now()
    delta_time = end_time - start_time
    runtimes.append(delta_time.total_seconds() * 1000)

In [ ]:
runtime_avg = round(np.mean(runtimes))
runtime_std = round(np.std(runtimes))

# Print a row of the table for the paper

In [ ]:
print("Registration PMMA block & Real image & " + 
      "{0:0.2f}".format(100 * MAPE) + "\\%    &    " +
      "{0:0.2f}".format(100 * ZNCC) + "\\%    &    " +
      "{0:0.2f}".format(SSIM) + "    &    $" +
      str(raw_x_ray_image.shape[1]) + " \\times " + str(raw_x_ray_image.shape[0]) + "$    &    " +
      str(number_of_triangles) + "    &    " +
      "N/A    &    " +
      "$" + str(runtime_avg) + " \\pm " + str(runtime_std) + "$ \\\\")